In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Count number of files in test set
import os
from pathlib import Path
from globals import TEST_DS_PATH

# Count number of files in test set
def count_files(path):
    return len(list(Path(path).glob("*")))

n_normal = count_files(f"{TEST_DS_PATH}/normal")
n_malignant = count_files(f"{TEST_DS_PATH}/malignant")
n_benign = count_files(f"{TEST_DS_PATH}/benign")

print("normal", n_normal)
print("malignant", n_malignant)
print("benign", n_benign)


normal 25
malignant 52
benign 79


In [3]:
import globals
import os
import pandas as pd

# Loads confusion matrices from each CV, calculates the mean and standard deviation
# and saves them to a file

name = "masked"
RANDOM_INIT_EXP_NAMES = [f"{name}-random-{seed}" for seed in globals.SEEDS]
paths = [os.path.join(globals.BASE_PATH, "experiments", n, f"conf_matrix_{n}.csv") for n in RANDOM_INIT_EXP_NAMES]

conf_matrices = [pd.read_csv(path, index_col=0) for path in paths]

mean_conf_matrix = pd.concat(conf_matrices).groupby(level=0).mean()
std_conf_matrix = pd.concat(conf_matrices).groupby(level=0).std()

mean_conf_matrix

,normal,malignant,benign
benign,0.001266,0.017722,0.981013
malignant,0.000000,0.936538,0.063462
normal,0.984000,0.004000,0.012000


In [4]:
std_conf_matrix

,normal,malignant,benign
benign,0.004003,0.013607,0.012302
malignant,0.000000,0.020372,0.020372
normal,0.027968,0.012649,0.026998


In [5]:
overall_mean_acc = (n_normal * mean_conf_matrix['normal']['normal'] + n_malignant * mean_conf_matrix['malignant']['malignant'] + n_benign * mean_conf_matrix['benign']['benign']) / (n_normal + n_malignant + n_benign)
overall_std_acc = (n_normal * std_conf_matrix['normal']['normal'] + n_malignant * std_conf_matrix['malignant']['malignant'] + n_benign * std_conf_matrix['benign']['benign']) / (n_normal + n_malignant + n_benign)

print("Overall mean accuracy:", overall_mean_acc)
print("Overall std accuracy:", overall_std_acc)

mean_conf_matrix.to_csv(f"../results/{name}_mean_conf_matrix.csv")
std_conf_matrix.to_csv(f"../results/{name}_std_conf_matrix.csv")

with open(f"../results/{name}_mean_acc.txt", "w") as f:
    f.write(str(overall_mean_acc))

with open(f"../results/{name}_std_acc.txt", "w") as f:
    f.write(str(overall_std_acc))

Overall mean accuracy: 0.9666666666666668
Overall std accuracy: 0.017502443077301467
